In [8]:
# 02_preprocess.ipynb  – Cell 1  (corrected)
from pathlib import Path
import numpy as np, torch, mne

DATA = Path.cwd() / "data"          # <─ remove .parent
RAW_FILE = DATA / "raw_20s.fif"     # or "raw_20s_raw.fif" if you renamed

assert RAW_FILE.exists(), f"Run 01_load_data first – {RAW_FILE} missing"
print("Found file:", RAW_FILE)

Found file: /Users/grantmckenzie/data/raw_20s.fif


In [9]:
# 02_preprocess.ipynb – Cell 2
raw = mne.io.read_raw_fif(RAW_FILE, preload=True, verbose=False)
print(raw)

<Raw | raw_20s.fif, 64 x 3201 (20.0 s), ~1.6 MiB, data loaded>


/var/folders/z4/3hnb86v562b2dpw2vfjywdtm0000gn/T/ipykernel_20307/245592129.py:2: RuntimeWarning: This filename (/Users/grantmckenzie/data/raw_20s.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(RAW_FILE, preload=True, verbose=False)


In [10]:
# 02_preprocess.ipynb – Cell 3
raw.set_eeg_reference("average", projection=True)      # common average ref
raw.notch_filter(60, verbose=False)                    # remove US power line
raw.filter(8, 30, fir_design="firwin", verbose=False)  # µ/β band (motor)
print("After filtering:", raw)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
After filtering: <Raw | raw_20s.fif, 64 x 3201 (20.0 s), ~1.6 MiB, data loaded>


In [11]:
# 02_preprocess.ipynb – Cell 4
EPOCH_LEN = 2.0        # seconds
epochs = mne.make_fixed_length_epochs(raw, duration=EPOCH_LEN,
                                      preload=True, verbose=False)
print("Epochs:", len(epochs), "shape", epochs.get_data().shape)

Epochs: 10 shape (10, 64, 320)


In [12]:
# 02_preprocess.ipynb – Cell 5
def spike_encode(epoch_arr: np.ndarray, thresh: float = 0.8):
    """
    Z-score each channel, threshold to spikes.
    epoch_arr: (n_ch, n_samples)  ->  return torch [T, C] binary
    """
    z = (epoch_arr - epoch_arr.mean(axis=1, keepdims=True)) / \
        epoch_arr.std(axis=1, keepdims=True)
    spk = (z > thresh).astype(np.float32)
    return torch.from_numpy(spk).T        # [T, C]

# quick sanity check with first epoch
test_tensor = spike_encode(epochs.get_data()[0])
print("Spike tensor shape =", test_tensor.shape,
      "sparsity =", test_tensor.mean().item())

Spike tensor shape = torch.Size([320, 64]) sparsity = 0.21650390326976776


In [13]:
# 02_preprocess.ipynb – Cell 6
spike_tensors = [spike_encode(e) for e in epochs.get_data()]

# Dummy labels (0) – replace with real event IDs later
labels = np.zeros(len(spike_tensors), dtype=np.int64)

print(f"Encoded {len(spike_tensors)} epochs →"
      f" tensor list of shape {[t.shape for t in spike_tensors[:1]][0]}")

Encoded 10 epochs → tensor list of shape torch.Size([320, 64])


In [14]:
# 02_preprocess.ipynb – Cell 7
OUT = DATA / "spikes_run4.pt"
torch.save({"spikes": spike_tensors,
            "labels": labels.tolist(),
            "fs": raw.info["sfreq"],
            "epoch_len_s": EPOCH_LEN},
           OUT)
print("Saved spike dataset →", OUT)

Saved spike dataset → /Users/grantmckenzie/data/spikes_run4.pt
